In [1]:
import os
from pathlib import Path
import pandas as pd
import sqlite3
import seaborn as sns

In [2]:
PROJECT_ROOT = Path(os.getcwd()).resolve().parents[0]

In [3]:
db_regional_path = PROJECT_ROOT / 'data/ONEE_Regional_COMPLETE_2007_2023.db'
db_regional = sqlite3.connect(db_regional_path)

In [ ]:
region = "Casablanca-Settat"
query = f"""
        SELECT
            Year as annee,
            AVG(GDP_Millions_DH) as pib_mdh,
            AVG(GDP_Primaire) as gdp_primaire,
            AVG(GDP_Secondaire) as gdp_secondaire,
            AVG(GDP_Tertiaire) as gdp_tertiaire,
            AVG(temp) as temperature_annuelle,
            AVG(Households_Total) as househods_total,
            AVG(Population_Total) as population_total
        FROM regional_features
        WHERE Region = '{region}'
        GROUP BY Year
    """

In [21]:
df_features = pd.read_sql_query(query, db_regional)

In [23]:
df_features

,annee,pib_mdh,gdp_primaire,gdp_secondaire,gdp_tertiaire,temperature_annuelle
0,2012,NaN,13535.0,87325.0,139139.0,NaN
1,2013,NaN,16126.0,93789.0,145503.0,18.744885
2,2014,NaN,12782.0,103327.0,145577.0,19.082672
3,2015,317925.0,17325.0,114662.0,151219.0,19.155323
4,2016,323919.0,14592.0,116719.0,158075.0,19.462040
5,2017,339720.0,18105.0,119520.0,164171.0,18.489678
6,2018,358404.0,19387.0,124955.0,172640.0,17.332142
7,2019,366211.0,14653.0,147225.0,187980.0,17.756119
8,2020,371368.0,11620.0,144742.0,172945.0,18.494009
9,2021,412252.0,19758.0,155434.0,186820.0,NaN


In [6]:
db_regional.close()

In [7]:
df.columns

Index(['City', 'Year', 'GDP_Millions_DH', 'Commerce', 'Construction',
       'Industrie', 'Services', 'Total', 'Households_Urban',
       'Households__Rural', 'Households_Total', 'Population_Urban',
       'Population_Rural', 'Population_Total', 'GDP_Primaire',
       'GDP_Secondaire', 'GDP_Tertiaire', 'Region', 'temp', 'tempmax',
       'tempmin', 'precip'],
      dtype='object')

In [8]:
df.isna().sum()

City                  0
Year                  0
GDP_Millions_DH      39
Commerce             39
Construction         39
Industrie            39
Services             39
Total                39
Households_Urban     55
Households__Rural    55
Households_Total     55
Population_Urban     55
Population_Rural     55
Population_Total     55
GDP_Primaire          0
GDP_Secondaire        0
GDP_Tertiaire         0
Region                0
temp                 34
tempmax              34
tempmin              34
precip               34
dtype: int64

In [ ]:
import numpy as np
import pandas as pd
from trend_classifier import classify_trend

y = 2.0 * np.exp(0.25 * np.arange(20)) + np.random.normal(scale=1.5, size=20)
result = classify_trend(y, plot=True)

print(result["best_model"])
print(result["score"])         # {'rss': ..., 'r2': ..., 'adj_r2': ..., 'aic': ...}
print(result["params"])        # model parameters


In [9]:
modeling_df = df[['Region', 'City', 'Year', "GDP_Millions_DH", "GDP_Primaire", "GDP_Secondaire", "GDP_Tertiaire", "temp"]]

In [20]:
modeling_df.to_csv("modeling_df.csv",index=False)

In [33]:
"""
Production-quality ARIMA modeling pipeline for multi-region, multi-variable time series.

This script performs end-to-end ARIMA modeling with proper train/test splits,
grid search for order selection, and comprehensive evaluation metrics.
"""

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from typing import Tuple, Dict, Optional, List
from dataclasses import dataclass
import logging
from tqdm import tqdm

from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import r2_score, mean_absolute_error

# ============================================================================
# CONFIGURATION
# ============================================================================

# Constants
TEST_HORIZON = 2
MIN_SERIES_LEN = TEST_HORIZON + 5  # Minimum 7 observations required
RANDOM_SEED = 42

# ARIMA grid search ranges
P_RANGE = range(0, 4)  # p in [0, 3]
D_RANGE = range(0, 3)  # d in [0, 2]
Q_RANGE = range(0, 4)  # q in [0, 3]

# Target variables
TARGET_VARIABLES = [
    "GDP_Millions_DH",
    "GDP_Primaire",
    "GDP_Secondaire",
    "GDP_Tertiaire",
    "temp"
]

# Output file
OUTPUT_FILE = "arima_results.xlsx"

# ============================================================================
# LOGGING SETUP
# ============================================================================

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# ============================================================================
# DATA STRUCTURES
# ============================================================================

@dataclass
class ModelResult:
    """Container for model results."""
    region: str
    variable: str
    n_train: int
    n_test: int
    order_p: int
    order_d: int
    order_q: int
    train_r2: float
    train_mae: float
    train_mape_pct: float
    test_r2: float
    test_mae: float
    test_mape_pct: float
    
@dataclass
class PredictionResult:
    """Container for predictions."""
    region: str
    variable: str
    year: int
    split: str
    y_true: float
    y_pred: float

# ============================================================================
# UTILITY FUNCTIONS
# ============================================================================

def calculate_mape(y_true: np.ndarray, y_pred: np.ndarray, epsilon: float = 1e-10) -> float:
    """
    Calculate Mean Absolute Percentage Error, handling zeros robustly.
    
    Args:
        y_true: True values
        y_pred: Predicted values
        epsilon: Small value to avoid division by zero
        
    Returns:
        MAPE as a percentage
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Exclude observations where true value is zero or very close to zero
    mask = np.abs(y_true) > epsilon
    
    if not mask.any():
        return np.nan
    
    mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    return mape

# ============================================================================
# CORE FUNCTIONS
# ============================================================================

def clean_series(df_region: pd.DataFrame, variable: str) -> Optional[pd.Series]:
    """
    Clean and prepare a time series for a specific region and variable.
    
    Steps:
    1. Sort by Year
    2. Drop leading/trailing NaNs first
    3. Interpolate internal missing values linearly
    4. Verify no NaNs remain
    5. Return series indexed by Year
    
    Args:
        df_region: DataFrame for a single region
        variable: Variable name to extract
        
    Returns:
        Cleaned series indexed by Year, or None if insufficient data
    """
    # Sort by year
    df_sorted = df_region.sort_values('Year').copy()
    
    # Extract the variable
    series = df_sorted.set_index('Year')[variable].copy()
    
    # Drop leading and trailing NaNs FIRST
    first_valid = series.first_valid_index()
    last_valid = series.last_valid_index()
    
    if first_valid is None or last_valid is None:
        # All values are NaN
        return None
    
    series = series.loc[first_valid:last_valid].copy()
    
    # Interpolate internal missing values
    if series.isna().any():
        series = series.interpolate(method='linear')
    
    # Final safety check - drop any remaining NaNs
    if series.isna().any():
        series = series.dropna()
    
    # Verify no NaNs remain
    if series.isna().any() or len(series) == 0:
        return None
    
    # Check minimum length
    if len(series) < MIN_SERIES_LEN:
        return None
    
    # Convert to float64 to ensure consistency
    series = series.astype(np.float64)
    
    return series


def split_train_test(y: pd.Series, horizon: int) -> Tuple[pd.Series, pd.Series]:
    """
    Split time series into train and test sets.
    
    Args:
        y: Time series
        horizon: Number of periods to hold out for testing
        
    Returns:
        Tuple of (y_train, y_test)
    """
    split_idx = len(y) - horizon
    y_train = y.iloc[:split_idx].copy()
    y_test = y.iloc[split_idx:].copy()
    
    return y_train, y_test


def select_arima_order(y_train: pd.Series) -> Optional[Tuple[int, int, int]]:
    """
    Select best ARIMA order using grid search on AIC.
    
    Args:
        y_train: Training time series (must be clean, no NaNs)
        
    Returns:
        Best order tuple (p, d, q) or None if all fail
    """
    # Verify input has no NaNs
    if y_train.isna().any():
        logger.error("y_train contains NaNs in select_arima_order")
        return None
    
    if len(y_train) < 3:  # Need at least 3 points for ARIMA
        return None
    
    best_aic = np.inf
    best_order = None
    
    # Convert to numpy array to avoid any pandas indexing issues
    y_values = y_train.values.astype(np.float64)
    
    for p in P_RANGE:
        for d in D_RANGE:
            for q in Q_RANGE:
                try:
                    model = ARIMA(y_values, order=(p, d, q))
                    fitted = model.fit()
                    
                    if fitted.aic < best_aic:
                        best_aic = fitted.aic
                        best_order = (p, d, q)
                        
                except Exception:
                    # Skip orders that fail to converge
                    continue
    
    return best_order


def fit_and_eval(
    y_train: pd.Series,
    y_test: pd.Series,
    order: Tuple[int, int, int]
) -> Optional[Dict]:
    """
    Fit ARIMA model and evaluate on train and test sets.
    
    Args:
        y_train: Training series (must be clean, no NaNs)
        y_test: Test series (must be clean, no NaNs)
        order: ARIMA order (p, d, q)
        
    Returns:
        Dictionary with metrics and predictions, or None if fitting fails
    """
    try:
        # Verify no NaNs in input
        if y_train.isna().any() or y_test.isna().any():
            logger.error(f"Input contains NaNs: train={y_train.isna().sum()}, test={y_test.isna().sum()}")
            return None
        
        if len(y_train) == 0 or len(y_test) == 0:
            return None
        
        # Convert to numpy arrays to avoid pandas issues
        y_train_values = y_train.values.astype(np.float64)
        y_test_values = y_test.values.astype(np.float64)
        
        # Fit model on training data
        model = ARIMA(y_train_values, order=order)
        fitted_model = model.fit()
        
        # In-sample predictions (training)
        train_pred_values = fitted_model.fittedvalues
        
        # ARIMA may drop initial observations due to differencing
        # Align predictions with original training data
        n_dropped = len(y_train_values) - len(train_pred_values)
        
        if n_dropped > 0:
            y_train_aligned = y_train.iloc[n_dropped:]
            train_pred = pd.Series(train_pred_values, index=y_train_aligned.index)
        else:
            y_train_aligned = y_train
            train_pred = pd.Series(train_pred_values, index=y_train.index)
        
        if len(y_train_aligned) == 0:
            return None
        
        # Out-of-sample forecast (test)
        forecast_values = fitted_model.forecast(steps=len(y_test))
        test_pred = pd.Series(forecast_values, index=y_test.index)
        
        # Calculate metrics for training set
        train_r2 = r2_score(y_train_aligned, train_pred)
        train_mae = mean_absolute_error(y_train_aligned, train_pred)
        train_mape = calculate_mape(y_train_aligned.values, train_pred.values)
        
        # Calculate metrics for test set
        test_r2 = r2_score(y_test, test_pred)
        test_mae = mean_absolute_error(y_test, test_pred)
        test_mape = calculate_mape(y_test.values, test_pred.values)
        
        return {
            'metrics': {
                'train_r2': train_r2,
                'train_mae': train_mae,
                'train_mape_pct': train_mape,
                'test_r2': test_r2,
                'test_mae': test_mae,
                'test_mape_pct': test_mape,
            },
            'predictions': {
                'train': (y_train_aligned, train_pred),
                'test': (y_test, test_pred)
            }
        }
        
    except Exception as e:
        logger.warning(f"Failed to fit ARIMA{order}: {str(e)}")
        return None


def process_region_variable(
    df: pd.DataFrame,
    region: str,
    variable: str
) -> Tuple[Optional[ModelResult], List[PredictionResult]]:
    """
    Process a single region-variable pair: clean, model, evaluate.
    
    Args:
        df: Full DataFrame
        region: Region name
        variable: Variable name
        
    Returns:
        Tuple of (ModelResult, list of PredictionResults) or (None, []) if skipped
    """
    # Filter to region
    df_region = df[df['Region'] == region].copy()
    
    # Clean series - this removes leading/trailing NaNs and interpolates internal ones
    y = clean_series(df_region, variable)
    
    if y is None:
        logger.warning(
            f"Skipping {region} × {variable}: insufficient data after cleaning "
            f"(< {MIN_SERIES_LEN} observations)"
        )
        return None, []
    
    # Split train/test
    y_train, y_test = split_train_test(y, TEST_HORIZON)
    
    # Select best ARIMA order
    best_order = select_arima_order(y_train)
    
    if best_order is None:
        logger.warning(
            f"Skipping {region} × {variable}: all ARIMA orders failed to fit"
        )
        return None, []
    
    # Fit and evaluate with best order
    result = fit_and_eval(y_train, y_test, best_order)
    
    if result is None:
        logger.warning(
            f"Skipping {region} × {variable}: final model fitting failed"
        )
        return None, []
    
    # Create ModelResult
    model_result = ModelResult(
        region=region,
        variable=variable,
        n_train=len(y_train),
        n_test=len(y_test),
        order_p=best_order[0],
        order_d=best_order[1],
        order_q=best_order[2],
        **result['metrics']
    )
    
    # Create PredictionResults
    prediction_results = []
    
    # Training predictions
    y_train_valid, train_pred = result['predictions']['train']
    for year, (true_val, pred_val) in zip(y_train_valid.index, zip(y_train_valid, train_pred)):
        prediction_results.append(
            PredictionResult(
                region=region,
                variable=variable,
                year=int(year),
                split='train',
                y_true=float(true_val),
                y_pred=float(pred_val)
            )
        )
    
    # Test predictions
    y_test_vals, test_pred = result['predictions']['test']
    for year, (true_val, pred_val) in zip(y_test_vals.index, zip(y_test_vals, test_pred)):
        prediction_results.append(
            PredictionResult(
                region=region,
                variable=variable,
                year=int(year),
                split='test',
                y_true=float(true_val),
                y_pred=float(pred_val)
            )
        )
    
    return model_result, prediction_results

# ============================================================================
# MAIN PIPELINE
# ============================================================================

def main():
    """Main execution pipeline."""
    
    # Set random seed
    np.random.seed(RANDOM_SEED)
    
    logger.info("=" * 80)
    logger.info("ARIMA MODELING PIPELINE")
    logger.info("=" * 80)
    
    # Load data
    logger.info("Loading data from modeling_df.csv...")
    modeling_df = pd.read_csv('modeling_df.csv')
    
    logger.info(f"Data shape: {modeling_df.shape}")
    logger.info(f"Regions: {modeling_df['Region'].nunique()}")
    logger.info(f"Variables to model: {len(TARGET_VARIABLES)}")
    
    # Get unique regions
    regions = modeling_df['Region'].unique()
    
    # Initialize result containers
    model_results = []
    prediction_results = []
    skipped_pairs = []
    
    # Process each region × variable pair
    total_pairs = len(regions) * len(TARGET_VARIABLES)
    logger.info(f"\nProcessing {total_pairs} region × variable pairs...")
    
    with tqdm(total=total_pairs, desc="Modeling progress") as pbar:
        for region in regions:
            for variable in TARGET_VARIABLES:
                pbar.set_description(f"{region} × {variable}")
                
                model_result, preds = process_region_variable(
                    modeling_df, region, variable
                )
                
                if model_result is not None:
                    model_results.append(model_result)
                    prediction_results.extend(preds)
                else:
                    skipped_pairs.append((region, variable))
                
                pbar.update(1)
    
    logger.info(f"\nSuccessfully modeled: {len(model_results)} pairs")
    logger.info(f"Skipped: {len(skipped_pairs)} pairs")
    
    # Convert results to DataFrames
    logger.info("\nCreating output DataFrames...")
    
    summary_df = pd.DataFrame([
        {
            'Region': r.region,
            'Variable': r.variable,
            'n_train': r.n_train,
            'n_test': r.n_test,
            'order_p': r.order_p,
            'order_d': r.order_d,
            'order_q': r.order_q,
            'train_R2': r.train_r2,
            'train_MAE': r.train_mae,
            'train_MAPE_pct': r.train_mape_pct,
            'test_R2': r.test_r2,
            'test_MAE': r.test_mae,
            'test_MAPE_pct': r.test_mape_pct,
        }
        for r in model_results
    ])
    
    predictions_df = pd.DataFrame([
        {
            'Region': p.region,
            'Variable': p.variable,
            'Year': p.year,
            'split': p.split,
            'y_true': p.y_true,
            'y_pred': p.y_pred,
        }
        for p in prediction_results
    ])
    
    # Save to Excel
    logger.info(f"\nSaving results to {OUTPUT_FILE}...")
    with pd.ExcelWriter(OUTPUT_FILE, engine='openpyxl') as writer:
        summary_df.to_excel(writer, sheet_name='summary', index=False)
        predictions_df.to_excel(writer, sheet_name='predictions', index=False)
    
    logger.info(f"Results saved successfully!")
    
    # Print summary report
    print("\n" + "=" * 80)
    print("MODELING SUMMARY REPORT")
    print("=" * 80)
    
    print(f"\nTotal pairs processed: {total_pairs}")
    print(f"Successfully modeled: {len(model_results)}")
    print(f"Skipped: {len(skipped_pairs)}")
    
    if skipped_pairs:
        print("\n--- SKIPPED PAIRS ---")
        for region, variable in skipped_pairs[:10]:  # Show first 10
            print(f"  • {region} × {variable}")
        if len(skipped_pairs) > 10:
            print(f"  ... and {len(skipped_pairs) - 10} more")
    
    if len(summary_df) > 0:
        print("\n--- TOP 10 MODELS BY TEST MAPE (BEST PERFORMANCE) ---")
        top_10 = summary_df.nsmallest(10, 'test_MAPE_pct')[
            ['Region', 'Variable', 'order_p', 'order_d', 'order_q', 
             'test_R2', 'test_MAE', 'test_MAPE_pct']
        ]
        print(top_10.to_string(index=False))
        
        print("\n--- OVERALL STATISTICS ---")
        print(f"Mean test R²: {summary_df['test_R2'].mean():.4f}")
        print(f"Mean test MAE: {summary_df['test_MAE'].mean():.2f}")
        print(f"Mean test MAPE: {summary_df['test_MAPE_pct'].mean():.2f}%")
        print(f"Median test MAPE: {summary_df['test_MAPE_pct'].median():.2f}%")
    
    print("\n" + "=" * 80)
    print(f"Pipeline complete! Results saved to: {OUTPUT_FILE}")
    print("=" * 80)


if __name__ == "__main__":
    main()

2025-10-29 12:56:53,047 - INFO - ================================================================================
2025-10-29 12:56:53,048 - INFO - ARIMA MODELING PIPELINE
2025-10-29 12:56:53,048 - INFO - ================================================================================
2025-10-29 12:56:53,049 - INFO - Loading data from modeling_df.csv...
2025-10-29 12:56:53,065 - INFO - Data shape: (119, 8)
2025-10-29 12:56:53,066 - INFO - Regions: 10
2025-10-29 12:56:53,067 - INFO - Variables to model: 5
2025-10-29 12:56:53,067 - INFO - 
Processing 50 region × variable pairs...
Tanger-Tétouan-Al Hoceïma × temp: 100%|██████████| 50/50 [02:47<00:00,  3.36s/it]           
2025-10-29 12:59:40,943 - INFO - 
Successfully modeled: 49 pairs
2025-10-29 12:59:40,944 - INFO - Skipped: 1 pairs
2025-10-29 12:59:40,945 - INFO - 
Creating output DataFrames...
2025-10-29 12:59:40,947 - INFO - 
Saving results to arima_results.xlsx...
2025-10-29 12:59:41,024 - INFO - Results saved successfully!



MODELING SUMMARY REPORT

Total pairs processed: 50
Successfully modeled: 49
Skipped: 1

--- SKIPPED PAIRS ---
  • Béni Mellal-Khénifra × temp

--- TOP 10 MODELS BY TEST MAPE (BEST PERFORMANCE) ---
                   Region       Variable  order_p  order_d  order_q     test_R2    test_MAE  test_MAPE_pct
           Marrakech-Safi           temp        0        2        2  -15.273437    0.122236       0.573837
     Béni Mellal-Khénifra  GDP_Tertiaire        0        2        1    0.437503  441.565907       1.260910
        Casablanca-Settat           temp        0        1        1  -35.994995    0.259464       1.382710
       Rabat-Salé-Kénitra           temp        0        0        1  -36.705734    0.350118       1.857805
              Souss-Massa           temp        0        1        0 -163.570947    0.408582       1.891642
               Fès-Meknès  GDP_Tertiaire        0        2        0    0.792253 1275.000000       1.935172
        Casablanca-Settat  GDP_Tertiaire        0    

In [36]:
modeling_df[modeling_df['Year'] == 2012]

,Region,City,Year,GDP_Millions_DH,GDP_Primaire,GDP_Secondaire,GDP_Tertiaire,temp
0,Souss-Massa,agadir,2012,NaN,10591.0,10975.0,26833.0,NaN
12,Béni Mellal-Khénifra,beni_mellal,2012,NaN,10736.0,26535.0,21509.0,NaN
24,Casablanca-Settat,casablanca,2012,NaN,13535.0,87325.0,139139.0,NaN
36,Drâa-Tafilalet,errachidia,2012,NaN,3332.0,3445.0,14653.0,NaN
48,Fès-Meknès,fes,2012,NaN,17352.0,17935.0,36842.0,NaN
60,Laâyoune-Sakia El Hamra,laayoune,2012,NaN,1051.0,5609.0,8245.0,NaN
72,Marrakech-Safi,marrakech,2012,NaN,13684.0,21696.0,40035.0,NaN
84,Oriental,oujda,2012,NaN,5122.0,8157.0,23103.0,NaN
95,Rabat-Salé-Kénitra,rabat,2012,NaN,17728.0,21135.0,92983.0,NaN
107,Tanger-Tétouan-Al Hoceïma,tanger,2012,NaN,9086.0,19334.0,35839.0,NaN


In [ ]:
query_bt = f"""
    SELECT
        Year as annee,
        Month as mois,
        Activity as activite,
        MWh * 1000 as consommation_kwh
    FROM monthly_data_bt
    WHERE Region = '{region}'
    ORDER BY Year, Month, Activity
"""

query_mt = f"""
    SELECT
        Year as annee,
        Month as mois,
        Activity as activite,
        MWh * 1000 as consommation_kwh
    FROM monthly_data_mt
    WHERE Region = '{region}'
    ORDER BY Year, Month, Activity
"""
    
df_bt = pd.read_sql_query(query_bt, db_regional)
df_mt = pd.read_sql_query(query_mt, db_regional)

df_mt['activite'] = df_mt['activite'].replace("Administratif", "Administratif_mt")
df = pd.concat([df_bt, df_mt], ignore_index=True)

In [27]:
df_annual = (
    df.groupby(['annee'])
    .agg({"consommation_kwh": 'sum'})
    .reset_index()
)

In [28]:
df_features

,annee,pib_mdh,gdp_primaire,gdp_secondaire,gdp_tertiaire,temperature_annuelle
0,2012,NaN,13535.0,87325.0,139139.0,NaN
1,2013,NaN,16126.0,93789.0,145503.0,18.744885
2,2014,NaN,12782.0,103327.0,145577.0,19.082672
3,2015,317925.0,17325.0,114662.0,151219.0,19.155323
4,2016,323919.0,14592.0,116719.0,158075.0,19.462040
5,2017,339720.0,18105.0,119520.0,164171.0,18.489678
6,2018,358404.0,19387.0,124955.0,172640.0,17.332142
7,2019,366211.0,14653.0,147225.0,187980.0,17.756119
8,2020,371368.0,11620.0,144742.0,172945.0,18.494009
9,2021,412252.0,19758.0,155434.0,186820.0,NaN


In [ ]:
import plotly.express as px

# Assuming df_annual is your DataFrame with columns "Year" and "MWh"
fig = px.line(df_annual, x="annee", y="consommation_kwh", title="Annual Energy Production (MWh)",
              markers=True)  # markers=True adds data points

fig = px.line(df_features, x="annee", y="pib_mdh", title="Annual Energy Production (MWh)",
              markers=True)  # markers=True adds data points

fig.update_layout(
    hovermode="x unified",  # unified hover label across traces
    xaxis_title="annee",
    template="plotly_white"
)

fig.show()
